In [1]:
import pandas as pd

## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

оценка 2 и ниже — низкий рейтинг;
оценка 4 и ниже — средний рейтинг;
оценка 4.5 и 5 — высокий рейтинг.
Результат классификации запишите в столбец class.

In [38]:
movies = pd.read_csv('D:/DataSci/Python/Numpy_pandas/hw3/ml-latest-small/ratings.csv')

In [39]:
def movie_classification(row):
    if row['rating'] <= 2:
        return 'Низкий рейтинг'
    elif row['rating'] <= 4:
        return 'Средний рейтинг'
    elif (row['rating'] == 4.5) or (row['rating'] == 5.0):
        return 'Высокий рейтинг'

In [40]:
movies['class'] = movies.apply(movie_classification, axis=1)

In [41]:
movies.head(10)

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,Средний рейтинг
1,1,1029,3.0,1260759179,Средний рейтинг
2,1,1061,3.0,1260759182,Средний рейтинг
3,1,1129,2.0,1260759185,Низкий рейтинг
4,1,1172,4.0,1260759205,Средний рейтинг
5,1,1263,2.0,1260759151,Низкий рейтинг
6,1,1287,2.0,1260759187,Низкий рейтинг
7,1,1293,2.0,1260759148,Низкий рейтинг
8,1,1339,3.5,1260759125,Средний рейтинг
9,1,1343,2.0,1260759131,Низкий рейтинг


## Задание 2
Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [42]:
geo_data = ({
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
})

keywords = pd.read_csv('D:/DataSci/Python/Numpy_pandas/hw3/ml-latest-small/keywords.csv')

#### Решение 1 
Ручное, передавая индексы (первое что пришло в голову, но мне оно не нравится) и судя по value_counts() еще и неверное

In [51]:
def geo_class(row):
    if (geo_data['Центр'][0] or geo_data['Центр'][1] or geo_data['Центр'][2]) in row['keyword'].lower():
        return 'Центр'
    if (geo_data['Северо-Запад'][0] or geo_data['Северо-Запад'][1] or geo_data['Северо-Запад'][2]) in row['keyword'].lower():
        return 'Северо-Запад'
    if (geo_data['Дальний Восток'][0] or geo_data['Дальний Восток'][1] or geo_data['Дальний Восток'][2]) in row['keyword'].lower():
        return 'Дальний Восток'
    return 'undefined'

In [52]:
%%time
keywords['region'] = keywords.apply(geo_class, axis=1)

Wall time: 980 ms


In [53]:
keywords['region'].value_counts()

undefined         99462
Центр               284
Северо-Запад        211
Дальний Восток       43
Name: region, dtype: int64

#### Решение 2
Чуть лучше, но тоже не восторг, накрутил костыли, еще и выполняется долго

In [54]:
new_geo_data = {}
for i in geo_data:
    for j in geo_data[i]:
        new_geo_data[j] = i

In [55]:
def geo_class_2(rows):
    for city, region in new_geo_data.items():
        if city.lower() in rows['keyword'].lower():
            return region
        
    return 'undefined'

In [56]:
%%time
keywords['region'] = keywords.apply(geo_class_2, axis=1)

Wall time: 2.35 s


In [57]:
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [58]:
keywords.sort_values(by='region', ascending=False).head(10)

,keyword,shows,region
6999,москва 24,37459,Центр
61986,такси везет москва официальный сайт,6063,Центр
43404,суперджоб москва вакансии,8208,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр
7090,гисметео ярославль,37008,Центр
97382,комсомольская правда москва,3708,Центр
35939,семейный доктор москва официальный сайт,9268,Центр
51916,сбербанк официальный сайт москва,7225,Центр
27302,расписание электричек тверь москва,23272,Центр


In [59]:
keywords.sort_values(by='region', ascending=False).tail(10)

,keyword,shows,region
33460,свет,9829,undefined
33459,аниме про демонов,9838,undefined
33458,df,9830,undefined
33457,конус,9830,undefined
33456,др,9830,undefined
33455,тобот мультфильм на русском смотреть онлайн,9790,undefined
33454,логические загадки,10369,undefined
33453,пенсионный фонд рф,9875,undefined
33452,olesya fattakhova,9832,undefined
99999,ггму,3630,undefined


#### Решение 3 
Лучшее что есть, немного взял помощи с Stackoverflow

In [26]:
def geo_classificator(data_row: str, geo: dict) -> str:
    for region, city_list in geo.items():
        if any(city in data_row for city in city_list):
            return region
        
    return 'undefined'

In [27]:
%%time
keywords['region'] = keywords['keyword'].apply(geo_classificator, geo=geo_data)

Wall time: 176 ms


In [28]:
keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

In [36]:
keywords.sort_values(by='region', ascending=False).head(10)

,keyword,shows,region
6999,москва 24,37459,Центр
61986,такси везет москва официальный сайт,6063,Центр
43404,суперджоб москва вакансии,8208,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр
7090,гисметео ярославль,37008,Центр
97382,комсомольская правда москва,3708,Центр
35939,семейный доктор москва официальный сайт,9268,Центр
51916,сбербанк официальный сайт москва,7225,Центр
27302,расписание электричек тверь москва,23272,Центр


In [37]:
keywords.sort_values(by='region', ascending=False).tail(10)

,keyword,shows,region
33460,свет,9829,undefined
33459,аниме про демонов,9838,undefined
33458,df,9830,undefined
33457,конус,9830,undefined
33456,др,9830,undefined
33455,тобот мультфильм на русском смотреть онлайн,9790,undefined
33454,логические загадки,10369,undefined
33453,пенсионный фонд рф,9875,undefined
33452,olesya fattakhova,9832,undefined
99999,ггму,3630,undefined
